In [1]:
# 加载训练数据

import polars as pl

In [2]:
train_data = pl.read_csv('../data/train.csv')
train_data

id,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,Fitness_Endurance-Season,Fitness_Endurance-Max_Stage,Fitness_Endurance-Time_Mins,Fitness_Endurance-Time_Sec,FGC-Season,FGC-FGC_CU,FGC-FGC_CU_Zone,FGC-FGC_GSND,FGC-FGC_GSND_Zone,FGC-FGC_GSD,FGC-FGC_GSD_Zone,FGC-FGC_PU,FGC-FGC_PU_Zone,FGC-FGC_SRL,FGC-FGC_SRL_Zone,FGC-FGC_SRR,FGC-FGC_SRR_Zone,FGC-FGC_TL,FGC-FGC_TL_Zone,BIA-Season,BIA-BIA_Activity_Level_num,BIA-BIA_BMC,BIA-BIA_BMI,…,BIA-BIA_ICW,BIA-BIA_LDM,BIA-BIA_LST,BIA-BIA_SMM,BIA-BIA_TBW,PAQ_A-Season,PAQ_A-PAQ_A_Total,PAQ_C-Season,PAQ_C-PAQ_C_Total,PCIAT-Season,PCIAT-PCIAT_01,PCIAT-PCIAT_02,PCIAT-PCIAT_03,PCIAT-PCIAT_04,PCIAT-PCIAT_05,PCIAT-PCIAT_06,PCIAT-PCIAT_07,PCIAT-PCIAT_08,PCIAT-PCIAT_09,PCIAT-PCIAT_10,PCIAT-PCIAT_11,PCIAT-PCIAT_12,PCIAT-PCIAT_13,PCIAT-PCIAT_14,PCIAT-PCIAT_15,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-Season,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
str,str,i64,i64,str,i64,str,f64,f64,f64,f64,i64,i64,i64,str,i64,i64,i64,str,i64,i64,f64,i64,f64,i64,i64,i64,f64,i64,f64,i64,f64,i64,str,i64,f64,f64,…,f64,f64,f64,f64,f64,str,f64,str,f64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,i64,str,i64,i64
"""00008ff9""","""Fall""",5,0,"""Winter""",51,"""Fall""",16.877316,46.0,50.8,null,null,null,null,null,null,null,null,"""Fall""",0,0,null,null,null,null,0,0,7.0,0,6.0,0,6.0,1,"""Fall""",2,2.66855,16.8792,…,24.4349,8.89536,38.9177,19.5413,32.6909,null,null,null,null,"""Fall""",5,4,4,0,4,0,0,4,0,0,4,0,4,4,4,4,4,4,2,4,55,null,null,null,"""Fall""",3,2
"""000fd460""","""Summer""",9,0,null,null,"""Fall""",14.03559,48.0,46.0,22.0,75,70,122,null,null,null,null,"""Fall""",3,0,null,null,null,null,5,0,11.0,1,11.0,1,3.0,0,"""Winter""",2,2.57949,14.0371,…,21.0352,14.974,39.4497,15.4107,27.0552,null,null,"""Fall""",2.34,"""Fall""",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""Fall""",46,64,"""Summer""",0,0
"""00105258""","""Summer""",10,1,"""Fall""",71,"""Fall""",16.648696,56.5,75.6,null,65,94,117,"""Fall""",5,7,33,"""Fall""",20,1,10.2,1,14.7,2,7,1,10.0,1,10.0,1,5.0,0,null,null,null,null,…,null,null,null,null,null,null,null,"""Summer""",2.17,"""Fall""",5,2,2,1,2,1,1,2,1,1,1,0,1,1,1,0,2,2,1,1,28,"""Fall""",38,54,"""Summer""",2,0
"""00115b9f""","""Winter""",9,0,"""Fall""",71,"""Summer""",18.292347,56.0,81.6,null,60,97,117,"""Summer""",6,9,37,"""Summer""",18,1,null,null,null,null,5,0,7.0,0,7.0,0,7.0,1,"""Summer""",3,3.84191,18.2943,…,30.4041,16.779,58.9338,26.4798,45.9966,null,null,"""Winter""",2.451,"""Summer""",4,2,4,0,5,1,0,3,2,2,3,0,3,0,0,3,4,3,4,1,44,"""Summer""",31,45,"""Winter""",0,1
"""0016bb22""","""Spring""",18,1,"""Summer""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,"""Summer""",1.04,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""ff8a2de4""","""Fall""",13,0,"""Spring""",60,"""Fall""",16.36246,59.5,82.4,null,71,70,104,null,null,null,null,"""Fall""",16,0,18.0,1,19.9,2,10,1,8.0,1,9.0,1,12.0,1,"""Fall""",3,4.52277,16.3642,…,33.3709,17.9797,66.2889,29.779,52.832,null,null,"""Winter""",3.26,"""Winter""",3,3,3,2,3,2,2,2,2,1,2,0,2,0,1,0,2,1,1,0,32,"""Winter""",35,50,"""Fall""",1,1
"""ffa9794a""","""Winter""",10,0,null,null,"""Spring""",18.764678,53.5,76.4,27.0,60,78,118,null,null,null,null,"""Spring""",0,0,null,null,null,null,4,0,0.0,0,0.0,0,12.0,1,"""Spring""",null,null,null,…,null,null,null,nul

观测数据可知，存在null值，所以需要对数据进行筛选或者填充。

In [ ]:
count = 0
for id in train_data['id']:
    try:
        file_load = pl.read_parquet(f'/kaggle/input/child-mind-institute-problematic-internet-use/series_train.parquet/id={id}/part-0.parquet')
    except:
        count+=1
count

观测得到，parquet文件并不是每个id都有的

查看null的比例，来判断是否应该填充

得到不同结果的数量，可知null的比例较大，不能舍去

In [37]:
(train_data
 .select(pl.col('PCIAT-PCIAT_Total'))
 .group_by(train_data.get_column('sii'))
 .agg(pl.col('PCIAT-PCIAT_Total').min().alias('PCIAT-PCIAT_Totalmin'),
      pl.col('PCIAT-PCIAT_Total').max().alias('PCIAT-PCIAT_Totalmax'),
      pl.col('PCIAT-PCIAT_Total').len().alias('count'))
      .sort('sii'))

sii,PCIAT-PCIAT_Totalmin,PCIAT-PCIAT_Totalmax,count
i64,i64,i64,u32
null,null,null,1224
0,0,30,1594
1,31,49,730
2,50,79,378
3,80,93,34


填充空白值
可选的填充方法
- KNN K近邻法
- 机器学习预测填充

In [71]:
import pandas as pd
train = pd.read_csv('../data/train.csv')

# 查看目标变量的缺失分布
print(train['sii'].isnull().sum())

# 分析其他特征与目标变量的相关性
correlations = train.corr()
print(correlations['sii'])


1224
Basic_Demos-Age                           0.365990
Basic_Demos-Sex                          -0.100148
CGAS-CGAS_Score                          -0.085261
Physical-BMI                              0.221125
Physical-Height                           0.373530
                                            ...   
PCIAT-PCIAT_Total                         0.899681
SDS-SDS_Total_Raw                         0.229692
SDS-SDS_Total_T                           0.232982
PreInt_EduHx-computerinternet_hoursday    0.336526
sii                                       1.000000
Name: sii, Length: 70, dtype: float64


C:\Users\Lenovo1\AppData\Local\Temp\ipykernel_13244\658698037.py:8: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  correlations = train.corr()


In [48]:
correlations

,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,...,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,sii
Basic_Demos-Age,1.000000,0.064004,-0.009994,0.492112,0.880274,0.748149,0.609601,0.107960,-0.218527,0.263875,...,0.085805,0.286888,0.159826,0.303259,0.286309,0.409559,0.037317,0.037115,0.441822,0.365990
Basic_Demos-Sex,0.064004,1.000000,0.075470,0.025793,-0.014454,0.003245,-0.020013,0.002076,0.026675,-0.025256,...,-0.097824,-0.102390,-0.110080,-0.104911,-0.067966,-0.093648,-0.020122,-0.018508,-0.000579,-0.100148
CGAS-CGAS_Score,-0.009994,0.075470,1.000000,-0.094074,-0.023185,-0.076399,-0.076116,-0.005292,-0.010108,-0.021392,...,-0.065687,-0.047770,-0.067922,-0.065210,-0.067920,-0.070542,-0.145369,-0.148457,-0.021596,-0.085261
Physical-BMI,0.492112,0.025793,-0.094074,1.000000,0.487097,0.865662,0.892149,0.141795,-0.036955,0.312045,...,0.071017,0.152122,0.100032,0.181208,0.161034,0.240858,0.087655,0.086185,0.296709,0.221125
Physical-Height,0.880274,-0.014454,-0.023185,0.487097,1.000000,0.833844,0.688033,0.115724,-0.217080,0.299157,...,0.112369,0.300590,0.184409,0.307084,0.292542,0.420765,0.036687,0.036478,0.399367,0.373530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PCIAT-PCIAT_Total,0.409559,-0.093648,-0.070542,0.240858,0.420765,0.353048,0.327013,0.069321,-0.037594,0.147081,...,0.769422,0.823708,0.802030,0.737243,0.754277,1.000000,0.234432,0.237718,0.374124,0.899681
SDS-SDS_Total_Raw,0.037317,-0.020122,-0.145369,0.087655,0.036687,0.075098,0.089831,0.006702,0.049194,0.036562,...,0.250015,0.177323,0.239387,0.171186,0.209167,0.234432,1.000000,0.996134,0.083501,0.229692
SDS-SDS_Total_T,0.037115,-0.018508,-0.148457,0.086185,0.036478,0.073482,0.091152,0.008502,0.047932,0.038317,...,0.252715,0.180220,0.241844,0.173366,0.209338,0.237718,0.996134,1.000000,0.086050,0.232982
PreInt_EduHx-computerinternet_hoursday,0.441822,-0.000579,-0.021596,0.296709,0.399367,0.375851,0.371418,0.074258,-0.073453,0.137728,...,0.180736,0.299658,0.223794,0.290488,0.245310,0.374124,0.083501,0.086050,1.000000,0.336526


采用均值填充等均不合理，首先采用 KNN 填充的方式

In [49]:
from sklearn.impute import SimpleImputer, KNNImputer

imputer = KNNImputer(n_neighbors=5)

numeric_cols = train.select_dtypes(include=['float64', 'int64']).columns

# 填充数据

imputed_data = imputer.fit_transform(train[numeric_cols])

train_imputed = pd.DataFrame(imputed_data, columns=numeric_cols)

train_imputed['sii'] = train_imputed['sii'].round().astype(int)



In [50]:
train_imputed

,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,...,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,sii
0,5.0,0.0,51.0,16.877316,46.00,50.8,23.0,61.2,86.4,110.6,...,4.0,4.0,4.0,2.0,4.0,55.0,48.4,62.2,3.0,2
1,9.0,0.0,70.0,14.035590,48.00,46.0,22.0,75.0,70.0,122.0,...,0.0,0.0,0.0,0.0,0.0,0.0,46.0,64.0,0.0,0
2,10.0,1.0,71.0,16.648696,56.50,75.6,24.8,65.0,94.0,117.0,...,0.0,2.0,2.0,1.0,1.0,28.0,38.0,54.0,2.0,0
3,9.0,0.0,71.0,18.292347,56.00,81.6,25.4,60.0,97.0,117.0,...,3.0,4.0,3.0,4.0,1.0,44.0,31.0,45.0,0.0,1
4,18.0,1.0,69.4,26.713639,64.14,125.0,33.6,70.6,74.2,125.8,...,2.6,2.0,1.6,1.6,1.4,40.0,42.0,58.8,2.6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3955,13.0,0.0,60.0,16.362460,59.50,82.4,25.0,71.0,70.0,104.0,...,0.0,2.0,1.0,1.0,0.0,32.0,35.0,50.0,1.0,1
3956,10.0,0.0,58.6,18.764678,53.50,76.4,27.0,60.0,78.0,118.0,...,0.8,2.0,0.8,0.8,0.4,22.4,38.6,54.8,0.0,0
3957,11.0,0.0,68.0,21.441500,60.00,109.8,28.6,79.0,99.0,116.0,...,0.0,1.0,1.0,0.0,1.0,31.0,56.0,77.0,0.0,1
3958,13.0,0.0,70.0,12.235895,70.70,87.0,27.6,59.0,61.0,113.0,...,1.0,1.0,1.0,1.0,1.0,19.0,33.0,47.0,1.0,0


In [51]:
for col in train.columns:
    if col not in numeric_cols:
        train_imputed[col] = train[col]


In [52]:
train_imputed

,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,Physical-Systolic_BP,...,CGAS-Season,Physical-Season,Fitness_Endurance-Season,FGC-Season,BIA-Season,PAQ_A-Season,PAQ_C-Season,PCIAT-Season,SDS-Season,PreInt_EduHx-Season
0,5.0,0.0,51.0,16.877316,46.00,50.8,23.0,61.2,86.4,110.6,...,Winter,Fall,NaN,Fall,Fall,NaN,NaN,Fall,NaN,Fall
1,9.0,0.0,70.0,14.035590,48.00,46.0,22.0,75.0,70.0,122.0,...,NaN,Fall,NaN,Fall,Winter,NaN,Fall,Fall,Fall,Summer
2,10.0,1.0,71.0,16.648696,56.50,75.6,24.8,65.0,94.0,117.0,...,Fall,Fall,Fall,Fall,NaN,NaN,Summer,Fall,Fall,Summer
3,9.0,0.0,71.0,18.292347,56.00,81.6,25.4,60.0,97.0,117.0,...,Fall,Summer,Summer,Summer,Summer,NaN,Winter,Summer,Summer,Winter
4,18.0,1.0,69.4,26.713639,64.14,125.0,33.6,70.6,74.2,125.8,...,Summer,NaN,NaN,NaN,NaN,Summer,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3955,13.0,0.0,60.0,16.362460,59.50,82.4,25.0,71.0,70.0,104.0,...,Spring,Fall,NaN,Fall,Fall,NaN,Winter,Winter,Winter,Fall
3956,10.0,0.0,58.6,18.764678,53.50,76.4,27.0,60.0,78.0,118.0,...,NaN,Spring,NaN,Spring,Spring,NaN,Winter,NaN,NaN,Winter
3957,11.0,0.0,68.0,21.441500,60.00,109.8,28.6,79.0,99.0,116.0,...,Spring,Winter,NaN,Winter,Winter,NaN,Winter,Winter,Winter,Fall
3958,13.0,0.0,70.0,12.235895,70.70,87.0,27.6,59.0,61.0,113.0,...,Spring,Winter,NaN,Spring,Summer,NaN,Spring,Spring,Spring,Spring


处理数据之后对数据进行编码

In [53]:
train.columns

Index(['id', 'Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex',
       'CGAS-Season', 'CGAS-CGAS_Score', 'Physical-Season', 'Physical-BMI',
       'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
       'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
       'Fitness_Endurance-Season', 'Fitness_Endurance-Max_Stage',
       'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
       'FGC-Season', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND',
       'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU',
       'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
       'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-Season',
       'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
       'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
       'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
       'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST'

season 数据 
两个：season of enrollment， season of participation， 貌似没用

In [55]:
data_description = pd.read_csv('../data/data_dictionary.csv')
data_description

,Instrument,Field,Description,Type,Values,Value Labels
0,Identifier,id,Participant's ID,str,NaN,NaN
1,Demographics,Basic_Demos-Enroll_Season,Season of enrollment,str,"Spring, Summer, Fall, Winter",NaN
2,Demographics,Basic_Demos-Age,Age of participant,float,NaN,NaN
3,Demographics,Basic_Demos-Sex,Sex of participant,categorical int,"0,1","0=Male, 1=Female"
4,Children's Global Assessment Scale,CGAS-Season,Season of participation,str,"Spring, Summer, Fall, Winter",NaN
...,...,...,...,...,...,...
76,Sleep Disturbance Scale,SDS-Season,Season of participation,str,"Spring, Summer, Fall, Winter",NaN
77,Sleep Disturbance Scale,SDS-SDS_Total_Raw,Total Raw Score,int,NaN,NaN
78,Sleep Disturbance Scale,SDS-SDS_Total_T,Total T-Score,int,NaN,NaN
79,Internet Use,PreInt_EduHx-Season,Season of participation,str,"Spring, Summer, Fall, Winter",NaN


预估重要的指标


In [56]:
correlations['sii']

Basic_Demos-Age                           0.365990
Basic_Demos-Sex                          -0.100148
CGAS-CGAS_Score                          -0.085261
Physical-BMI                              0.221125
Physical-Height                           0.373530
                                            ...   
PCIAT-PCIAT_Total                         0.899681
SDS-SDS_Total_Raw                         0.229692
SDS-SDS_Total_T                           0.232982
PreInt_EduHx-computerinternet_hoursday    0.336526
sii                                       1.000000
Name: sii, Length: 70, dtype: float64

In [57]:
correlations['PCIAT-PCIAT_Total']

Basic_Demos-Age                           0.409559
Basic_Demos-Sex                          -0.093648
CGAS-CGAS_Score                          -0.070542
Physical-BMI                              0.240858
Physical-Height                           0.420765
                                            ...   
PCIAT-PCIAT_Total                         1.000000
SDS-SDS_Total_Raw                         0.234432
SDS-SDS_Total_T                           0.237718
PreInt_EduHx-computerinternet_hoursday    0.374124
sii                                       0.899681
Name: PCIAT-PCIAT_Total, Length: 70, dtype: float64

In [61]:

sii_corr = correlations['PCIAT-PCIAT_Total'].drop('PCIAT-PCIAT_Total')
filtered_corr = sii_corr[(sii_corr > 0.1) | (sii_corr < -0.1)]
filtered_corr

Basic_Demos-Age                           0.409559
Physical-BMI                              0.240858
Physical-Height                           0.420765
Physical-Weight                           0.353048
Physical-Waist_Circumference              0.327013
Physical-Systolic_BP                      0.147081
FGC-FGC_CU                                0.287494
FGC-FGC_GSND                              0.146813
FGC-FGC_GSD                               0.160472
FGC-FGC_PU                                0.196006
FGC-FGC_SRL_Zone                         -0.148850
FGC-FGC_SRR_Zone                         -0.109682
FGC-FGC_TL                                0.136696
BIA-BIA_BMI                               0.248060
BIA-BIA_FFMI                              0.109694
BIA-BIA_Frame_num                         0.193631
PCIAT-PCIAT_01                            0.727959
PCIAT-PCIAT_02                            0.788802
PCIAT-PCIAT_03                            0.823336
PCIAT-PCIAT_04                 

相关性较强的指标
Basic_Demos-Age                           0.409559
Physical-BMI                              0.240858
Physical-Height                           0.420765
Physical-Weight                           0.353048
Physical-Waist_Circumference              0.327013
Physical-Systolic_BP                      0.147081
FGC-FGC_CU                                0.287494
FGC-FGC_GSND                              0.146813
FGC-FGC_GSD                               0.160472
FGC-FGC_PU                                0.196006
FGC-FGC_SRL_Zone                         -0.148850
FGC-FGC_SRR_Zone                         -0.109682
FGC-FGC_TL                                0.136696
BIA-BIA_BMI                               0.248060
BIA-BIA_FFMI                              0.109694
BIA-BIA_Frame_num                         0.193631

In [63]:
season_cols = [col for col in train.columns if 'Season' in col]
filter_data = train.drop(season_cols, axis=1)

In [64]:
filter_data

,id,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,...,PCIAT-PCIAT_16,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-computerinternet_hoursday,sii
0,00008ff9,5,0,51.0,16.877316,46.0,50.8,NaN,NaN,NaN,...,4.0,4.0,4.0,2.0,4.0,55.0,NaN,NaN,3.0,2.0
1,000fd460,9,0,NaN,14.035590,48.0,46.0,22.0,75.0,70.0,...,0.0,0.0,0.0,0.0,0.0,0.0,46.0,64.0,0.0,0.0
2,00105258,10,1,71.0,16.648696,56.5,75.6,NaN,65.0,94.0,...,0.0,2.0,2.0,1.0,1.0,28.0,38.0,54.0,2.0,0.0
3,00115b9f,9,0,71.0,18.292347,56.0,81.6,NaN,60.0,97.0,...,3.0,4.0,3.0,4.0,1.0,44.0,31.0,45.0,0.0,1.0
4,0016bb22,18,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3955,ff8a2de4,13,0,60.0,16.362460,59.5,82.4,NaN,71.0,70.0,...,0.0,2.0,1.0,1.0,0.0,32.0,35.0,50.0,1.0,1.0
3956,ffa9794a,10,0,NaN,18.764678,53.5,76.4,27.0,60.0,78.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
3957,ffcd4dbd,11,0,68.0,21.441500,60.0,109.8,NaN,79.0,99.0,...,0.0,1.0,1.0,0.0,1.0,31.0,56.0,77.0,0.0,1.0
3958,ffed1dd5,13,0,70.0,12.235895,70.7,87.0,NaN,59.0,61.0,...,1.0,1.0,1.0,1.0,1.0,19.0,33.0,47.0,1.0,0.0


In [65]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [66]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols)
    ]
)
X = preprocessor.fit_transform(filter_data)

In [68]:
X.shape

(3960, 70)

In [70]:
import numpy as np

import pandas as pd

import os

import re

from sklearn.base import clone

from sklearn.metrics import cohen_kappa_score

from sklearn.model_selection import StratifiedKFold

from scipy.optimize import minimize

from concurrent.futures import ThreadPoolExecutor

from tqdm import tqdm

import polars as pl

import polars.selectors as cs

import matplotlib.pyplot as plt

from matplotlib.ticker import MaxNLocator, FormatStrFormatter, PercentFormatter

import seaborn as sns



from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt

from keras.models import Model

from keras.layers import Input, Dense

from keras.optimizers import Adam

import torch

import torch.nn as nn

import torch.optim as optim



from colorama import Fore, Style

from IPython.display import clear_output

import warnings

from lightgbm import LGBMRegressor

from xgboost import XGBRegressor

from catboost import CatBoostRegressor

from sklearn.ensemble import VotingRegressor, RandomForestRegressor, GradientBoostingRegressor

from sklearn.impute import SimpleImputer, KNNImputer

from sklearn.pipeline import Pipeline

warnings.filterwarnings('ignore')

pd.options.display.max_columns = None

ModuleNotFoundError: No module named 'keras'

In [69]:
if np.any(np.isinf(train)):

    train = train.replace([np.inf, -np.inf], np.nan)



def quadratic_weighted_kappa(y_true, y_pred):

    return cohen_kappa_score(y_true, y_pred, weights='quadratic')



def threshold_Rounder(oof_non_rounded, thresholds):

    return np.where(oof_non_rounded < thresholds[0], 0,

                    np.where(oof_non_rounded < thresholds[1], 1,

                             np.where(oof_non_rounded < thresholds[2], 2, 3)))



def evaluate_predictions(thresholds, y_true, oof_non_rounded):

    rounded_p = threshold_Rounder(oof_non_rounded, thresholds)

    return -quadratic_weighted_kappa(y_true, rounded_p)

NameError: name 'np' is not defined